### Goal: Generate Images and labels for training

In [74]:
import pandas as pd

In [75]:
filepath = r"C:\Users\ken\Desktop\OneDrive\OneDrive - Knights - University of Central Florida\UCF Spring 2019\Machine Learning\data"
filename = r"\Yahoo_S&P500_2004-2019.csv"

In [76]:
df = pd.read_csv(filepath+filename)
#df["date"] = pd.to_datetime(df["date"])
df["date"] = pd.to_datetime(df["Date"])
df = df.set_index('date')

In [77]:
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
date,,,,,,,
2004-01-02,2004-01-02,1111.920044,1118.849976,1105.079956,1108.479980,1108.479980,1153200000
2004-01-05,2004-01-05,1108.479980,1122.219971,1108.479980,1122.219971,1122.219971,1578200000
2004-01-06,2004-01-06,1122.219971,1124.459961,1118.439941,1123.670044,1123.670044,1494500000
2004-01-07,2004-01-07,1123.670044,1126.329956,1116.449951,1126.329956,1126.329956,1704900000
2004-01-08,2004-01-08,1126.329956,1131.920044,1124.910034,1131.920044,1131.920044,1868400000


In [78]:
 df.shape

(3816, 7)

In [95]:
import numpy as np

def to_sequences(seq_size, seq_size_out, obs):
    x = []
    y = []
    y_label = []
    
    for i in range(len(obs)-seq_size-seq_size_out-1):
        window = obs[i:(i+seq_size)]
        after_window = obs[(i+seq_size):(i+seq_size+seq_size_out)]
        window = [[x] for x in window]
        after_window = [[y] for y in after_window]
        
        x.append(window)
        y.append(after_window)
        
        
        average = 0
        for j in range(seq_size_out):
            #print( y[i][j][0][4])
            average += y[i][j][0][4]
        
        average = average/seq_size_out
        y_label.append(average)
        
    return np.array(x), np.array(y),np.array(y_label)

In [96]:
x, y, yMean = to_sequences(90, 3,df.values)

In [97]:
yMean

array([1096.473307  , 1092.079956  , 1090.42997233, ..., 2787.88663733,
       2794.22664367, 2794.129964  ])

In [107]:
"""
    Generates a list of values which corresponds to 
    sell - 0
    hold - 1
    buy  - 2
    
    Algorithm
    Latest price + percentThreshold% > predicted yMean Upward movement : Action buy
    Predicted yMean < 1% gain or loss : Action hold
    Latest price + percentThreshold% > predicted yMean Downward movement : Action sell
"""
def threshhold(x, yMean, percentThreshold):
    size = len(yMean)
    y = []
    y_close = []
    for i in range(size):
        percentVal = x[i][-1][0][4] * percentThreshold
        change = yMean[i] - x[i][-1][0][4]
        y_close.append(x[i][-1][0][4])
        #Positive change means price went up and negative means price went down
        if(abs(change) > percentVal):
            if(change > 0):
                y.append(2)
            else:
                y.append(0)
        else:
            y.append(1)

    return np.array(y), np.array(y_close)

In [124]:
y_new, y_close = threshhold(x, yMean, .01)

In [125]:
x.shape

(3722, 90, 1, 7)

In [126]:
test = x[0].reshape(90,7)
test.shape

(90, 7)

In [127]:
columns = ['date','open','high','low','close','adj close', 'volume']
df = pd.DataFrame(test,columns=columns)
df["date"] = pd.to_datetime(df["date"])
df.head()

,date,open,high,low,close,adj close,volume
0,2004-01-02,1111.92,1118.85,1105.08,1108.48,1108.48,1153200000
1,2004-01-05,1108.48,1122.22,1108.48,1122.22,1122.22,1578200000
2,2004-01-06,1122.22,1124.46,1118.44,1123.67,1123.67,1494500000
3,2004-01-07,1123.67,1126.33,1116.45,1126.33,1126.33,1704900000
4,2004-01-08,1126.33,1131.92,1124.91,1131.92,1131.92,1868400000


In [128]:
x[0][-1][0][0]
dates = []
for i in range(x.shape[0]):
    dates.append(x[i][-1][0][0])
    
dates = np.array(dates)

In [135]:
y_df = pd.DataFrame()
y_df['label'] = y_new

In [136]:
y_df['dates'] = dates
y_df['close'] = y_close
y_df['future_mean'] = yMean
y_df.to_csv(r'E:\Data\S&P500\label_90out_3in_3pctchange.csv')

In [137]:
y_df[:]

,label,dates,close,future_mean
0,1,2004-05-11,1095.449951,1096.473307
1,1,2004-05-12,1097.280029,1092.079956
2,1,2004-05-13,1096.439941,1090.429972
3,1,2004-05-14,1095.699951,1088.090007
4,1,2004-05-17,1084.099976,1089.786662
5,1,2004-05-18,1091.489990,1090.476685
6,1,2004-05-19,1088.680054,1092.720011
7,2,2004-05-20,1089.189941,1100.673381
8,2,2004-05-21,1093.560059,1107.800008
9,2,2004-05-24,1095.410034,1116.423340


In [24]:
from math import pi
import pandas as pd
from bokeh.plotting import figure, show, output_file
from bokeh.io import export_png

def createAndSavePlot(df, location = r"E:\Data\S&P500", filename = r"\plot.png", plot_height=100,plot_width=200):
    inc = df.close > df.open
    dec = df.open > df.close
    w = 12*60*60*1000 # half day in ms

    #TOOLS = "pan,wheel_zoom,box_zoom,reset,save"

    p = figure(x_axis_type="datetime", plot_height=100,plot_width=200)
    #p.xaxis.major_label_orientation = pi/4
    #p.grid.grid_line_alpha=0.3

    p.segment(df.date, df.high, df.date, df.low, color="black")
    p.vbar(df.date[inc], w, df.open[inc], df.close[inc], fill_color="#D5E1DD", line_color="black")
    p.vbar(df.date[dec], w, df.open[dec], df.close[dec], fill_color="#F2583E", line_color="black")

    p.toolbar.logo = None
    p.toolbar_location = None

    p.xaxis.major_tick_line_color = None  # turn off x-axis major ticks
    p.xaxis.minor_tick_line_color = None  # turn off x-axis minor ticks
    p.yaxis.major_tick_line_color = None  # turn off y-axis major ticks
    p.yaxis.minor_tick_line_color = None  # turn off y-axis minor ticks
    p.xaxis.major_label_text_color = None  #note that this leaves space between the axis and the axis label  
    p.yaxis.major_label_text_color = None  #note that this leaves space between the axis and the axis label

    export_png(p, filename= location+filename)

In [26]:
location = r"E:\Data\S&P500\train_images_90in_3out"

for i in range(y_df.shape[0]):
    temp = x[i].reshape(90,7)
    columns = ['date','open','high','low','close','adj close', 'volume']
    df = pd.DataFrame(temp,columns=columns)
    df["date"] = pd.to_datetime(df["date"])
        
    filename= r"\{}.png".format(str(df.iloc[-1].date)[:10])
    createAndSavePlot(df, location,filename)

In [19]:
inc = df.close > df.open
dec = df.open > df.close
w = 12*60*60*1000 # half day in ms

#TOOLS = "pan,wheel_zoom,box_zoom,reset,save"

p = figure(x_axis_type="datetime", plot_height=100,plot_width=200)
#p.xaxis.major_label_orientation = pi/4
#p.grid.grid_line_alpha=0.3

p.segment(df.date, df.high, df.date, df.low, color="black")
p.vbar(df.date[inc], w, df.open[inc], df.close[inc], fill_color="#D5E1DD", line_color="black")
p.vbar(df.date[dec], w, df.open[dec], df.close[dec], fill_color="#F2583E", line_color="black")

p.toolbar.logo = None
p.toolbar_location = None

p.xaxis.major_tick_line_color = None  # turn off x-axis major ticks
p.xaxis.minor_tick_line_color = None  # turn off x-axis minor ticks
p.yaxis.major_tick_line_color = None  # turn off y-axis major ticks
p.yaxis.minor_tick_line_color = None  # turn off y-axis minor ticks
p.xaxis.major_label_text_color = None  #note that this leaves space between the axis and the axis label  
p.yaxis.major_label_text_color = None  #note that this leaves space between the axis and the axis label

export_png(p, filename="plot.png")

'C:\\Users\\ken\\Desktop\\OneDrive\\OneDrive - Knights - University of Central Florida\\UCF Spring 2019\\Machine Learning\\signal\\plot.png'